In [1]:
from neo4j import GraphDatabase, basic_auth
import torch
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience
from neo4j import basic_auth
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


c:\Users\GameToGo\Documents\Vscode\SMM_hw1\smm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from neo4j import GraphDatabase, basic_auth
url = 'bolt://localhost:7687'
user = 'neo4j'
password = 'asdf1234'

driver = GraphDatabase.driver(url, auth=basic_auth(user, password))

def fetch_data(query, params={}):
  with driver.session() as session:
    result = session.run(query, params)
    return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [3]:
# Override NEO4J_URI and NEO4J_AUTH here according to your setup
url = 'bolt://localhost:7687'
user = 'neo4j'
password = 'asdf1234'
gds = GraphDataScience(url, auth=basic_auth(user, password))

# # Necessary if you enabled Arrow on the db. Override with your DB name
# NEO4J_DB = "neo4j"
# gds.set_database(NEO4J_DB)

In [15]:
def get_pd(column: str) -> pd.DataFrame:
    if(column=='Card'):
        r = 'HAS_CC'
    elif(column=='IP'):
        r = 'HAS_IP'
    elif(column=='Device'):
        r = 'USED'
    query = f"""
    MATCH (u:User)-[r:{r}]->(c:{column})
    WITH u, c, c.degree AS degree
    ORDER BY id(u), degree DESC
    WITH u, collect(id(c)) AS {column}_ids, collect(degree) AS degrees
    RETURN id(u) AS user_id, {column}_ids[0..5] AS top5_{column}
    """
    df = fetch_data(query)
    return df

In [16]:
user = fetch_data("""
MATCH (u:User)
WITH u
ORDER BY id(u)
RETURN id(u) AS user_id, u.fraudRisk AS fraud_risk
""")
user

,user_id,fraud_risk
0,12220793,0
1,12220820,0
2,12220855,0
3,12220870,0
4,12220879,0
...,...,...
33727,14883929,0
33728,14884036,0
33729,14884202,0
33730,14884214,0


In [7]:
def transform_serial(df, name:str)-> pd.DataFrame:
    df = df.explode(f'top5_{name}')
    # Reset the index
    df = df.reset_index(drop=True)
    serial_numbers = df[f'top5_{name}'].astype(str)  # Convert to string type if necessary
    mapped_numbers, _ = pd.factorize(serial_numbers)
    df['MappedSerialNumber'] = mapped_numbers
    grouped = df.groupby('user_id')['MappedSerialNumber'].apply(list).reset_index()

    # Create separate columns for each card
    cards_df = pd.DataFrame(grouped['MappedSerialNumber'].to_list(), columns=[f'{name}{i}' for i in range(1,6)])
    merged_df = pd.concat([grouped['user_id'], cards_df], axis=1)
    for i in range(1,6):
        merged_df[f'{name}{i}'] = merged_df[f'{name}{i}'].fillna(-1).astype(int)
    return merged_df

In [17]:
ip = get_pd('IP')
device = get_pd('Device')
card = get_pd('Card')


In [18]:
card = transform_serial(card, 'Card')
ip = transform_serial(ip, 'IP')
device = transform_serial(device, 'Device')

In [19]:
user = user.merge(card, on='user_id', how='outer')
user = user.merge(ip, on='user_id', how='outer')
user = user.merge(device, on='user_id', how='outer')

In [20]:
for i in ['Card', 'IP', 'Device']:
    for j in range(1,6):
        user[f'{i}{j}'] = user[f'{i}{j}'].fillna(-1).astype(int)

In [21]:
user_dict = user['user_id'].to_dict()
inv_user_dict = {id:index  for index, id in user_dict.items()}
user['user_id'] = user['user_id'].map(inv_user_dict)

In [22]:
user

,user_id,fraud_risk,Card1,Card2,Card3,Card4,Card5,IP1,IP2,IP3,IP4,IP5,Device1,Device2,Device3,Device4,Device5
0,0,0,0,1,2,-1,-1,0,1,2,3,4,0,1,2,3,-1
1,1,0,3,-1,-1,-1,-1,5,6,7,8,9,4,-1,-1,-1,-1
2,2,0,4,5,-1,-1,-1,10,11,12,13,14,5,-1,-1,-1,-1
3,3,0,6,7,8,9,10,15,16,17,18,19,6,7,-1,-1,-1
4,4,0,11,12,13,14,15,16,17,18,19,20,8,9,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33727,33727,0,90325,90326,-1,-1,-1,605,1983,2844,5172,2955,3168,49287,-1,-1,-1
33728,33728,0,75608,75606,75607,90327,-1,1543,24653,24654,5142,24655,49288,49289,49290,49291,-1
33729,33729,0,90328,90329,-1,-1,-1,90,162,2067,1952,2754,-1,-1,-1,-1,-1
33730,33730,0,90330,-1,-1,-1,-1,326,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [23]:
edge = fetch_data("""
MATCH (u1:User)-[r:P2P]->(u2:User)
RETURN id(u1), id(u2)
""")
edge

,id(u1),id(u2)
0,12220793,14625332
1,12220793,14625332
2,12220820,13601992
3,12220870,13312545
4,12220870,13631046
...,...,...
102827,14884202,14869808
102828,14884214,14808309
102829,14884248,14653861
102830,14884248,14653861


In [24]:
edge['id(u1)'] = edge['id(u1)'].map(inv_user_dict)
edge['id(u2)'] = edge['id(u2)'].map(inv_user_dict)
edge

,id(u1),id(u2)
0,0,29106
1,0,29106
2,1,10314
3,3,4645
4,3,10933
...,...,...
102827,33729,33515
102828,33730,32573
102829,33731,29828
102830,33731,29828


In [26]:
user.to_csv('node_info.csv', index=False)

In [27]:
edge.to_csv('edge_index.csv', index=False)

# Appendix

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv, to_hetero

from torch_geometric.data import HeteroData
from torch_geometric.transforms import ToUndirected, RandomLinkSplit

In [ ]:
from torch_geometric.nn.conv import GCNConv
import torch.nn as nn
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        # 初始化self.convs, self.bns, and self.softmax. 請記得參考Pytorch文檔。
        # Initialize self.convs, self.bns, and self.softmax. please refer to Pytorch document
        # num_layers為GNN層數
        # num_layers is the number of GNN layer

        super(GCN, self).__init__()
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        self.convs.append(GCNConv(input_dim, hidden_dim))
        self.bns.append(nn.BatchNorm1d(hidden_dim))
        for i in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
            self.bns.append(nn.BatchNorm1d(hidden_dim))
        self.convs.append(GCNConv(hidden_dim, output_dim))
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = dropout

        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        # 輸入x為node feature tensor，adj_t為edge index tensor，請回傳output tensor，計算過程參考上方圖片
        # Input x is node feature tensor，adj_t is edge index tensor，please return output tensor, calculation prcess can be refernce from the figure on top.

        out = None

        ############# CODE HERE ############
        ## 1. 請嚴格按照上圖說明進行神經網絡建構
        ## 2. 可以使用F.relu和F.dropout：https://pytorch.org/docs/stable/nn.functional.html
        ## 3. 請透過self.training設置F.dropout的使用時機（self.training==True時才進行dropout）
        ## 1. Please follow the above instructions strictly for the construction of the neural network
        ## 2. you can use F.relu and F.dropout: https://pytorch.org/docs/stable/nn.functional.html
        ## 3. Please set the use of F.dropout through self.training (dropout will be performed only when self.training==True)
        for i in range(len(self.convs) - 1):
            x = self.convs[i](x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            if self.training == True:
                x = F.dropout(x, p=self.dropout)
        x = self.convs[-1](x, adj_t)
        x = self.softmax(x)
        out = x
        #########################################

        return out

In [ ]:
def train(model, data, train_idx, optimizer, loss_fn):
    # 使用傳入的optimizer和loss_fn來訓練模型
    # Use the input optimizer and loss_fn to train the model
    # 此處不需使用batch概念
    # you don't need the concept of batch here
    model.train()
    loss = 0

    ############# CODE HERE ############
    optimizer.zero_grad()
    x, edge_index = data.x, data.adj_t
    output = model(x, edge_index)
    y_pred = output
    y = data.y.view(-1)
    loss = loss_fn(y_pred, y)
    #########################################

    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
# Test function
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    # 使用split_idx和evaluator評估模型表現
    # please use split_idx adn evaluator to evaluate the model
    model.eval()

    # The output of model on all data
    out = None

    ############# CODE HERE ############
    ## 1. 完成out的運算即可，記得將輸入資料轉移至device上
    ## 1. Just get output using model and remember to transfer the input data to the device
    x, edge_index = data.x, data.adj_t
    out = model(x, edge_index)
    #########################################

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [ ]:
# 注意：請勿修改此處參數！
# Don't change!
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
args

In [ ]:
model = GCN(15, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
# evaluator = Evaluator(name='ogbn-arxiv')

In [ ]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, optimizer, loss_fn)
  result = test(model, data, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

In [ ]:
# gds.run_cypher('''
# MATCH (c:Card)
# WITH c
# ORDER BY c.guid
# WITH COLLECT(c) AS cards, range(0, size(collect(c))-1) AS ids
# UNWIND cards AS card
# UNWIND ids AS indexs
# SET card.id = indexs
# ''')

In [ ]:
ip = fetch_data("""
MATCH (u:FraudRiskUser)-[r:HAS_IP]->(i:IP)
WITH u, i, i.degree AS degree
ORDER BY id(u), degree DESC
WITH u, collect(id(i)) AS card_ids, collect(degree) AS degrees
RETURN id(u) AS user_id, card_ids[0..5] AS top5_ips
""")
ip

In [ ]:
fetch_data("""
MATCH (u:FraudRiskUser)-[r:HAS_CC]->(c:Card)
WITH u, c, c.degree AS degree
ORDER BY id(u), degree DESC
WITH u, collect(id(c)) AS card_ids, collect(degree) AS degrees
UNWIND card_ids[0..5] AS top_card_ids
RETURN id(u) AS user_id, top_card_ids, degrees[0..4] AS top_card_degrees
""")

In [ ]:
def expand_df(df, property):
    expanded_df = df[property].apply(pd.Series)
    # Rename the expanded columns
    expanded_df.columns = [f'{property}_{i+1}' for i in range(expanded_df.shape[1])]

    # Concatenate the expanded columns with the original DataFrame
    result_df = pd.concat([df, expanded_df], axis=1)

    # Drop the original 'top_card_ids' column
    result_df.drop(property, axis=1, inplace=True)
    return result_df

In [ ]:
device = expand_df(device, 'top5_Device')
card = expand_df(card, 'top5_Card')
ip = expand_df(ip, 'top5_IP')
user = pd.concat([user, ip, card, device], axis=1)

In [24]:
def load_node(cypher, index_col, encoders=None, **kwargs):
    # Execute the cypher query and retrieve data from Neo4j
    df = fetch_data(cypher)
    df.set_index(index_col, inplace=True)
    # Define node mapping
    mapping = {index: i for i, index in enumerate(df.index.unique())}
    # Define node features
    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [25]:
def load_edge(cypher, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    # Execute the cypher query and retrieve data from Neo4j
    df = fetch_data(cypher)
    # Define edge index
    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])
    # Define edge features
    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [ ]:
class SequenceEncoder(object):
    # The 'SequenceEncoder' encodes raw column strings into embeddings.
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [ ]:
class GenresEncoder(object):
    # The 'GenreEncoder' splits the raw column strings by 'sep' and converts
    # individual elements to categorical labels.
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x

In [ ]:
class IdentityEncoder(object):
    # The 'IdentityEncoder' takes the raw column values and converts them to
    # PyTorch tensors.
    def __init__(self, dtype=None, is_list=False):
        self.dtype = dtype
        self.is_list = is_list

    def __call__(self, df):
        if self.is_list:
            return torch.stack([torch.tensor(el) for el in df.values])
        return torch.from_numpy(df.values).to(self.dtype)

In [ ]:
user_query = """
MATCH (u:Merchant) RETURN u.id AS userId
"""

user_x, user_mapping = load_node(user_query, index_col='userId')

In [ ]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

In [ ]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "neo4j://44.208.167.239:7687",
  auth=basic_auth("neo4j", "torpedoes-tube-resistance"))

cypher_query = '''
MATCH (m:Movie {title:$movie})<-[:RATED]-(u:User)-[:RATED]->(rec:Movie)
RETURN distinct rec.title AS recommendation LIMIT 20
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      movie="Crimson Tide").data())
  print(results)
  for record in results:
    print(record['recommendation'])

driver.close()

In [1]:
import pandas as pd